## Comparing jaxtronomy and lenstronomy performance for lens model ray shooting

In [ ]:
from jax import numpy as jnp, config, block_until_ready
import numpy as np
import time

from jaxtronomy.LensModel.profile_list_base import lens_class, _JAXXED_MODELS as JAXXED_DEFLECTOR_PROFILES
from lenstronomy.LensModel.profile_list_base import lens_class as lens_class_lenstronomy

config.update("jax_enable_x64", True)

num_pix = 180

x_jax = jnp.tile(jnp.linspace(-5.0, 5.0, num_pix) + 100, num_pix)
y_jax = jnp.repeat(jnp.linspace(-5.0, 5.0, num_pix) + 100, num_pix)

x = np.tile(np.linspace(-5.0, 5.0, num_pix) + 100, num_pix)
y = np.repeat(np.linspace(-5.0, 5.0, num_pix) + 100, num_pix)

for deflector_profile in JAXXED_DEFLECTOR_PROFILES:
    # Skip these profiles
    if deflector_profile in ["LOS", "LOS_MINIMAL"]:
        continue

    profile_lenstronomy = lens_class_lenstronomy(deflector_profile)
    profile = lens_class(deflector_profile)

    # Get parameter names
    kwargs_lens = profile.upper_limit_default

    # Compile code/warmup
    f_x, f_y = profile_lenstronomy.derivatives(x, y, **kwargs_lens)
    f_x, f_y = profile.derivatives(x_jax, y_jax, **kwargs_lens)
    block_until_ready(f_x)
    block_until_ready(f_y)
    

    # Now time runtime after compilation/warmup
    start_time = time.perf_counter()
    for _ in range(10000):
        f_x, f_y = profile.derivatives(x_jax, y_jax, **kwargs_lens)
        block_until_ready(f_x)
        block_until_ready(f_y)

    middle_time = time.perf_counter()

    for _ in range(10000):
        f_x, f_y = profile_lenstronomy.derivatives(x, y, **kwargs_lens)

    end_time = time.perf_counter()

    jax_execution_time = middle_time - start_time
    lenstronomy_execution_time = end_time - middle_time
    print(
        f"jaxtronomy execution time for {deflector_profile}: {jax_execution_time} seconds"
    )
    print(
        f"lenstronomy execution time for {deflector_profile}: {lenstronomy_execution_time} seconds"
    )
    print(
        f"jaxtronomy is {'{0:.1f}'.format(lenstronomy_execution_time/jax_execution_time)}x faster\n"
    )

    # Additional performance comparison with EPL_NUMBA
    if deflector_profile == "EPL":
        profile_lenstronomy = lens_class_lenstronomy("EPL_NUMBA")
        f_x, f_y = profile_lenstronomy.derivatives(x, y, **kwargs_lens)
        start_time = time.perf_counter()
        for _ in range(10000):
            f_x, f_y = profile_lenstronomy.derivatives(x, y, **kwargs_lens)
        end_time = time.perf_counter()
        numba_execution_time = end_time - start_time

        print(f"jaxtronomy execution time for EPL: {jax_execution_time} seconds")
        print(
            f"lenstronomy execution time for EPL_NUMBA: {numba_execution_time} seconds"
        )
        print(
            f"jaxtronomy is {'{0:.1f}'.format(numba_execution_time/jax_execution_time)}x faster\n"
        )

## Comparing jaxtronomy and lenstronomy performance for light model surface brightness
Note: Restarting the kernel is recommended before running the below tests on a low-memory machine, otherwise you may introduce a memory bottleneck from caching all of the compiled functions from the lens model testing in the previous cell

In [ ]:
import copy
from jax import numpy as jnp, block_until_ready
import numpy as np
import time

from jaxtronomy.LightModel.light_model import LightModel
from lenstronomy.LightModel.light_model import LightModel as LightModel_ref
from jaxtronomy.LightModel.light_model_base import (
    _JAXXED_MODELS as JAXXED_SOURCE_PROFILES,
)

# 60x60 grid
num_pix = 180

x_jax = jnp.tile(jnp.linspace(-5.0, 5.0, num_pix) + 100, num_pix)
y_jax = jnp.repeat(jnp.linspace(-5.0, 5.0, num_pix) + 100, num_pix)

x = np.tile(np.linspace(-5.0, 5.0, num_pix) + 100, num_pix)
y = np.repeat(np.linspace(-5.0, 5.0, num_pix) + 100, num_pix)

for source_profile in JAXXED_SOURCE_PROFILES:

    # For testing different n_max in the SHAPELETS profile
    # if source_profile != "SHAPELETS":
    #     continue

    lightModel = LightModel([source_profile])
    lightModel_ref = LightModel_ref([source_profile])
    kwargs_source = copy.deepcopy(lightModel.func_list[0].upper_limit_default)
    if source_profile in ["MULTI_GAUSSIAN", "MULTI_GAUSSIAN_ELLIPSE"]:
        kwargs_source["amp"] = np.linspace(10, 20, 5)
        kwargs_source["sigma"] = np.linspace(0.3, 1.0, 5)
    elif source_profile == "SHAPELETS":
        n_max = 6
        num_param = int((n_max + 1) * (n_max + 2) / 2)
        kwargs_source["n_max"] = n_max
        kwargs_source["amp"] = np.linspace(20.0, 30.0, num_param)

    # Compile code/warmup
    flux_jax = lightModel.func_list[0].function(x_jax, y_jax, **kwargs_source)
    block_until_ready(flux_jax)
    
    flux = lightModel_ref.func_list[0].function(x, y, **kwargs_source)

    # Now time runtime after compilation/warmup
    start_time = time.perf_counter()
    for _ in range(10000):
        flux_jax = lightModel.func_list[0].function(x_jax, y_jax, **kwargs_source)
        block_until_ready(flux_jax)

    middle_time = time.perf_counter()

    for _ in range(10000):
        flux = lightModel_ref.func_list[0].function(x, y, **kwargs_source)

    end_time = time.perf_counter()

    jax_execution_time = middle_time - start_time
    lenstronomy_execution_time = end_time - middle_time
    print(
        f"jaxtronomy execution time for {source_profile}: {jax_execution_time} seconds"
    )
    print(
        f"lenstronomy execution time for {source_profile}: {lenstronomy_execution_time} seconds"
    )
    print(
        f"jaxtronomy is {'{0:.1f}'.format(lenstronomy_execution_time/jax_execution_time)}x faster\n"
    )

## Comparing jaxtronomy and lenstronomy performance for image convolution
We start by comparing Gaussian convolution. Due to the fact that there is no jax.scipy.ndimage.gaussian_filter function, in jaxtronomy we implement gaussian convolution by constructing a gaussian kernel and performing a pixel kernel fft convolution. This is done in such a way to match scipy.ndimage.guassian_filter with mode="nearest" as closely as possible. The jaxtronomy implementation is slower, but the benefit of this JAX implementation is that we can use autodifferentiation for model fitting.

In [ ]:
from lenstronomy.ImSim.Numerics.convolution import MultiGaussianConvolution
from jaxtronomy.ImSim.Numerics.convolution import GaussianConvolution

from jax import numpy as jnp
import numpy as np
import time

num_pix = 60
sigma = 0.5
pixel_scale = 0.11
supersampling_factor = 3
supersampling_convolution = False

for truncation in range(1, 10):

    jax_conv = GaussianConvolution(
        sigma, pixel_scale, supersampling_factor, supersampling_convolution, truncation
    )
    lenstronomy_conv = MultiGaussianConvolution(
        [sigma],
        [1],
        pixel_scale,
        supersampling_factor,
        supersampling_convolution,
        truncation,
    )

    kernel_radius = round(sigma * truncation / pixel_scale)
    if supersampling_convolution:
        kernel_radius *= supersampling_factor

    image_jax = jnp.tile(jnp.linspace(1.0, 100.0, num_pix), num_pix).reshape(
        (num_pix, num_pix)
    )
    image = np.tile(np.linspace(1.0, 100.0, num_pix), num_pix).reshape(
        (num_pix, num_pix)
    )

    # Compile code/warmup
    result_jax = jax_conv.convolution2d(image_jax)
    result = lenstronomy_conv.convolution2d(image)
    np.testing.assert_allclose(result_jax, result, rtol=1e-5, atol=1e-5)

    # Now time runtime after compilation/warmup
    start_time = time.perf_counter()

    for _ in range(10000):
        jax_conv.convolution2d(image_jax)

    middle_time = time.perf_counter()

    for _ in range(10000):
        lenstronomy_conv.convolution2d(image)

    end_time = time.perf_counter()

    jax_execution_time = middle_time - start_time
    lenstronomy_execution_time = end_time - middle_time
    print(
        f"jaxtronomy execution time for gaussian convolution with kernel radius {kernel_radius}: {jax_execution_time} seconds"
    )
    print(
        f"lenstronomy execution time for gaussian convolution with kernel radius {kernel_radius}: {lenstronomy_execution_time} seconds"
    )
    print(
        f"lenstronomy is {'{0:.1f}'.format(jax_execution_time/lenstronomy_execution_time)}x faster\n"
    )

Now we compare fft convolution with a given kernel. Generally, jax.scipy.signal.fftconvolve is about twice as fast as the normal scipy version, but there are some specific kernel sizes where the jax version is slower than usual for some unknown reason.

In [ ]:
from lenstronomy.ImSim.Numerics.convolution import (
    PixelKernelConvolution as PixelKernelConvolution_ref,
)
from jaxtronomy.ImSim.Numerics.convolution import PixelKernelConvolution

from jax import numpy as jnp
import numpy as np
import time

num_pix = 60
kernel_size_list = np.linspace(3, 45, 22, dtype=int)

supersampling_factor = 3
supersampling_convolution = False

if supersampling_convolution:
    num_pix *= supersampling_factor
    kernel_size_list *= supersampling_factor

for kernel_size in kernel_size_list:
    kernel_jax = jnp.tile(jnp.linspace(0.9, 1.0, kernel_size), kernel_size).reshape(
        (kernel_size, kernel_size)
    )
    kernel_jax = kernel_jax / jnp.sum(kernel_jax)
    kernel = np.tile(np.linspace(0.9, 1.0, kernel_size), kernel_size).reshape(
        (kernel_size, kernel_size)
    )
    kernel = kernel / np.sum(kernel)

    jax_conv = PixelKernelConvolution(kernel=kernel_jax, convolution_type="fft")
    lenstronomy_conv = PixelKernelConvolution_ref(kernel=kernel, convolution_type="fft")

    image_jax = jnp.tile(jnp.linspace(1.0, 20.0, num_pix), num_pix).reshape(
        (num_pix, num_pix)
    )
    image = np.tile(np.linspace(1.0, 20.0, num_pix), num_pix).reshape(
        (num_pix, num_pix)
    )

    # Compile code/warmup
    result_jax = jax_conv.convolution2d(image_jax)
    result = lenstronomy_conv.convolution2d(image)
    np.testing.assert_allclose(result_jax, result, rtol=1e-5, atol=1e-5)

    # Now time runtime after compilation/warmup
    start_time = time.perf_counter()

    for _ in range(10000):
        jax_conv.convolution2d(image_jax)

    middle_time = time.perf_counter()

    for _ in range(10000):
        lenstronomy_conv.convolution2d(image)

    end_time = time.perf_counter()

    jax_execution_time = middle_time - start_time
    lenstronomy_execution_time = end_time - middle_time
    print(
        f"jaxtronomy execution time for fft convolution with kernel size {kernel_size}: {jax_execution_time} seconds"
    )
    print(
        f"lenstronomy execution time for fft convolution with kernel size {kernel_size}: {lenstronomy_execution_time} seconds"
    )
    print(
        f"jaxtronomy is {'{0:.1f}'.format(lenstronomy_execution_time/jax_execution_time)}x faster\n"
    )